# Sleep data analysis notebook

In [ ]:
%matplotlib inline
import graph

import glob
import numpy as np
import pandas as pd
import datetime as dt

from scipy.interpolate import interp1d
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from matplotlib import dates as mdates, pyplot as plt

DAY_SECONDS = 24 * 60 * 60
TIMEZONE = 1 * 60 * 60
COLUMNS = {'active': 'General Activity', 'vc_0': 'Web Status', 'vc_8': 'FB App Status', 'vc_74': 'Messenger Status', 'vc_10':'Other Status', 'type': 'Activity Type'}

### Read data and display statistics

In [ ]:
# read data
names = glob.glob('./generated_graphs/csv/*.csv')
dataframes = [pd.read_csv(f).rename(columns=COLUMNS).dropna() for f in names]
    
USERS = len(dataframes)

for i in range(USERS):
    dataframes[i]['time'] += TIMEZONE

print("Loaded data gathered from {0} people".format(USERS))

start = np.min(pd.concat(dataframes)['time'].dropna(0))
end = np.max(pd.concat(dataframes)['time'].dropna(0))

print('First registered data point at {0} local time\nLast registered data point at {1} local time'.format(dt.datetime.utcfromtimestamp(start), dt.datetime.utcfromtimestamp(end)))

for i in range(USERS):
    dataframes[i]['time'] -= start
    
end = np.max(pd.concat(dataframes)['time'])
print('Gathered data over {0}'.format(dt.datetime.utcfromtimestamp(end - DAY_SECONDS).strftime('%d days, %H:%M')))

### Interpolate data to get timeseries of set width

In [ ]:
# default timeseries width is 2 minutes
SERIES_WIDTH = 2 * 60
POINTS_IN_DAY = np.round(DAY_SECONDS / SERIES_WIDTH).astype(int)

In [ ]:
series = []
timeseries = np.arange(0, end, SERIES_WIDTH)

for i in range(USERS):
    interpolate = interp1d(dataframes[i]['time'], dataframes[i].iloc[:, 1:], axis=0, kind='nearest', bounds_error=False, fill_value=np.zeros(dataframes[i].shape[1] - 1))
    df = pd.DataFrame(columns=dataframes[0].columns)
    df['time'] = timeseries
    df.iloc[:, 1:] = np.round(interpolate(timeseries))
    series.append(df)
    
series[0].head()

In [ ]:
interesting = ['General Activity', 'Messenger Status', 'FB App Status', 'Web Status']

dataset = dataframes[0]
events = [dataset[dataset[key] == 1]['time'] for key in interesting]

dataset = series[0]
events_series = [dataset[dataset[key] == 1]['time'] for key in interesting]

fig = plt.figure(figsize=[20, 5])

ax = fig.add_subplot(1, 2, 1)
for (event, place, color) in zip(events, range(4), ['blue', 'orange', 'green', 'red']):
    ax.eventplot(event, color=color, lineoffsets=place, linewidths = 0.5)
ax.set_title('Gathered data eventplot')
ax.set_xlabel('Timestamp')
ax.legend(interesting)

ax = fig.add_subplot(1, 2, 2)
for (event, place, color) in zip(events_series, range(4), ['blue', 'orange', 'green', 'red']):
    ax.eventplot(event, color=color, lineoffsets=place, linewidths = 0.5)
ax.set_title('Interpolated timeseries eventplot')
ax.set_xlabel('Timestamp')
ax.legend(interesting)

plt.show()

### Reshape data to get individual days

In [ ]:
DAYS = np.floor(end / DAY_SECONDS).astype(int)
FEATURES = len(dataframes[0].columns)

data = np.zeros((USERS, DAYS, POINTS_IN_DAY, FEATURES))

t = dt.datetime.utcfromtimestamp(start)
t2 = t.hour * 3600 + t.minute * 60 + t.second

for i in range(USERS):
    for j in range(DAYS):
        data[i, j] = series[i].iloc[j * POINTS_IN_DAY : (j + 1) * POINTS_IN_DAY]
        data[i, j, :, 0] = (data[i, j, :, 0] + t2) % DAY_SECONDS
        
print('Final dataset shape: {0}'.format(data.shape))
print('Sample (user 0, day 0):\n{0}'.format(data[0, 0]))

In [ ]:
average = pd.DataFrame(columns=dataframes[0].columns)
average['time'] = data[0, 0, :, 0]

av = np.zeros((POINTS_IN_DAY, FEATURES - 1))
for i in range(USERS):
    for j in range(DAYS):
        av += data[i, j, :, 1:]
        
av /= (USERS * DAYS)

average.iloc[:, 1:] = av
print('{0} full days worth of data collected'.format(DAYS))
average.describe()

### Plot individual features of datasets

In [ ]:
def plot_feature(input_dataset, feature):
    assert input_dataset.shape == (POINTS_IN_DAY, FEATURES)
    dataset = input_dataset.sort_values('time')
    
    plt.figure(figsize=(20,10))

    # activity
    ax1 = plt.subplot(211)
    xs = [dt.datetime.utcfromtimestamp(t) for t in dataset['time']]
    ax1.stackplot(xs, dataset[feature].transpose())

    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.gca().xaxis.set_major_locator(mdates.HourLocator())
    plt.gcf().autofmt_xdate()
    plt.legend([feature])
    plt.xlabel('Time during the day')
    
plot_feature(average, 'Messenger Status')

### Plot everyday activity habits

In [ ]:
def plot_habits(dataset, bounds='auto'):
    daily = False
    if dataset.shape[0] == POINTS_IN_DAY:
        daily = True
        dataset = dataset.sort_values('time')
    
    plt.figure(figsize=(20,10))

    # activity
    ax1 = plt.subplot(211)
    xs = [dt.datetime.utcfromtimestamp(t) for t in dataset['time']]
    ax1.stackplot(xs, dataset['General Activity'].transpose())
    
    if daily:
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        plt.gca().xaxis.set_major_locator(mdates.HourLocator())
    else:
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d'))
        plt.gca().xaxis.set_major_locator(mdates.DayLocator())
    
    plt.gcf().autofmt_xdate()
    plt.legend([dataset.columns[1]])
    
    if bounds != 'auto':
        plt.ylim(bounds)

    # divided into classes
    ax2 = plt.subplot(212, sharex=ax1)
    ax2.stackplot(xs, dataset[['Web Status', 'FB App Status', 'Other Status', 'Messenger Status']].transpose())

    if daily:
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        plt.gca().xaxis.set_major_locator(mdates.HourLocator())
    else:
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mdates.DayLocator())
    
    plt.gcf().autofmt_xdate()
    
    if bounds != 'auto':
        plt.ylim(bounds)

    plt.legend(dataset.columns[2:-1])
    plt.xlabel('Time')
    plt.show()
    
def find_name(name):
    for i in range(USERS):
        if name in names[i]:
            return i
    raise Exception('User {0} not found!'.format(name))

def user_average(name):
    i = find_name(name)

    average = pd.DataFrame(columns=dataframes[0].columns)
    average['time'] = data[i, 0, :, 0].astype(int)

    av = np.zeros((POINTS_IN_DAY, FEATURES - 1))
    for j in range(DAYS):
        av += data[i, j, :, 1:] / DAYS

    average.iloc[:, 1:] = av
    return average

In [ ]:
plot_habits(average)

In [ ]:
plot_habits(user_average('#REDACTED#'))

### PCA dimensionality reduction and outlier detection

In [ ]:
n_components = 2

daily = np.mean(data, axis=1).reshape((USERS, POINTS_IN_DAY * FEATURES))
pca = PCA(n_components, svd_solver='full')
pca.fit(daily)

daily_pca = pca.transform(daily)

print('Data reduced from {0} to {1} dimensions'.format(POINTS_IN_DAY * FEATURES, pca.n_components_))
size_ratio = pca.n_components_ / (POINTS_IN_DAY * FEATURES)
print('{:d}x reduction in size ({:.2%} original), {:.2%} variance'.format(np.floor(1 / size_ratio).astype(int), size_ratio, np.sum(pca.explained_variance_ratio_)))

In [ ]:
target = '#REDACTED#'
index = find_name(target)

km = KMeans(n_clusters = 6)
clusters = km.fit_predict(daily)

fig = plt.figure(figsize=(8, 5))
plt.scatter(daily_pca[:, 0], daily_pca[:, 1], c=clusters, marker='.')
plt.scatter(daily_pca[index, 0], daily_pca[index, 1], c='red', marker='x', s=75)
plt.title('{0} - dimensional PCA, {1}-means clustering'.format(pca.n_components_, km.n_clusters))
plt.legend(['everyone', target])
plt.show()

In [ ]:
def user_data(name):
    i = find_name(name)

    ret = pd.DataFrame(columns=dataframes[0].columns)
    ret['time'] = timeseries + start
    ret.iloc[:, 1:] = series[i]
    ret['Day Time'] = (timeseries + t2) % DAY_SECONDS

    return ret

outlier = user_data('#REDACTED#')
plot_habits(outlier)

### PCA latent space sampling

In [ ]:
from ipywidgets import *

def sample_PCA(pca, **kwargs):
    vector = np.zeros(pca.n_components_)
    i = 0
    for value in kwargs.values():
        vector[i] = value
        i += 1
    
    inverse = pca.inverse_transform(vector).reshape((POINTS_IN_DAY, FEATURES))
    average = pd.DataFrame(columns=dataframes[0].columns)
    average['time'] = inverse[:, 0]
    average.iloc[:, 1:] = inverse[:, 1:]
    plot_habits(average, [-0.1, 0.5])
    
def interactivePCA(dims, person=False):
    pca = PCA(n_components=dims, svd_solver='full')
    pca.fit(daily)
    dims = pca.n_components_
    if not person:
        vals = np.zeros(dims)
    else:
        vals = pca.transform(daily[find_name(person)].reshape((1, POINTS_IN_DAY * FEATURES))).flatten()

    sliders = [widgets.FloatSlider(value=vals[i], min=-6, max=6, step=1, description='feature ' + str(i), orientation='vertical') for i in range(dims)]
    kwargs = dict(zip(['feature_' + str(t) for t in range(dims)], sliders))
    out = interactive_output(lambda **kwargs: sample_PCA(pca, **kwargs), kwargs)
    display(VBox([HBox(sliders), out]))
        
interactivePCA(12, '#REDACTED#')

### Predicting user's sleep patterns for incoming days
This doesn't work as of now, will try again when more data is collected

In [ ]:
from keras.models import Sequential
from keras.layers import Input, SimpleRNN, LSTM, Dense, Activation
from keras.losses import MSE
from sklearn.preprocessing import StandardScaler

In [ ]:
data_train = user_data('#REDACTED#')
X_train = data_train.drop(columns=['time', 'Other Status', 'Activity Type'])

hours = 2
width = np.floor(hours * 3600 / SERIES_WIDTH).astype(int)

n, m = X_train.shape
n -= width

def generate_series(X, width):
    k, _ = X.shape
    for i in range(k - width):
        yield X.iloc[i:i + width, :]
        
def generate_next(X, width):
    k, _ = X.shape
    for i in range(width, k):
        yield X.iloc[i, :-1]
        
X = np.reshape([np.array(t) for t in generate_series(X_train, width)], (n, width * m))

std = StandardScaler().fit(X)
X = np.reshape(std.transform(X), (n, width, m))

y = np.reshape([np.array(t) for t in generate_next(X_train, width)], (n, m - 1))

print('X shape: {0}\ny shape: {1}'.format(X.shape, y.shape))

model = Sequential()

model.add(SimpleRNN(128, use_bias=True, return_sequences=False, input_shape=(width, m)))
model.add(Dense(m - 1, use_bias=True))
model.add(Activation('sigmoid'))
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])

history = model.fit(X, y, batch_size=512, epochs=100)

In [ ]:
model.evaluate(X, y)

In [ ]:
fig = plt.figure(figsize=(20, 5))

ax = fig.add_subplot(121)
ax.plot(history.history['loss'])

plt.title('Learning curve')
plt.xlabel('Iteration')
plt.ylabel('Loss')

ax = fig.add_subplot(122)
ax.plot(history.history['acc'])

plt.title('Accuracy')
plt.xlabel('Iteration')
plt.ylabel('Accuracy')

vals = ax.get_yticks()
ax.set_yticklabels(['{:.0%}'.format(x) for x in vals])

plt.show()

In [ ]:
tmp = pd.DataFrame(columns=data_train.columns)
tmp['time'] = data_train.loc[width:, 'time']
tmp.iloc[:, [1, 2, 3, 5]] = y
tmp = tmp.fillna(0)

plot_habits(tmp)

In [ ]:
y_pred = model.predict(X)

def mse(y_pred, y_true):
    return np.mean(np.power(y_pred - y_true, 2))

print('Not rounded:', mse(y_pred, y))
print('Rounded:', mse(np.round(y_pred), y))

In [ ]:
pred_df = pd.DataFrame(columns=data_train.columns)
pred_df['time'] = data_train.loc[width:, 'time']
pred_df.iloc[:, [1, 2, 3, 5]] = np.round(y_pred)
pred_df = pred_df.fillna(0)

plot_habits(pred_df)

In [ ]:
def sample_model(model, start, start_index, samples = 200):
    width, _ = model.get_input_shape_at(0)[1:]
    X = np.array(start.iloc[start_index:start_index + width, [1, 2, 3, 5, 7]])
    
    for _ in range(samples):
        yield X[0]
        p = model.predict(std.transform(X.reshape((1, width * m))).reshape((1, width, m)))
        X = np.roll(X, -1, axis=0)
        X[-1, : m - 1] = np.round(p)
        X[-1, -1] = (X[-2, -1] + SERIES_WIDTH) % DAY_SECONDS

samples = 2 * width
start = user_data('#REDACTED#')
start_index = np.random.randint(0, start.shape[0] - width)

new = np.array([g for g in sample_model(model, start, start_index, samples)])

gen_df = pd.DataFrame(columns=data_train.columns)
gen_df['time'] = start.loc[start_index:start_index + samples - 1, 'time']
gen_df.iloc[:, [1,2,3,5,7]] = np.reshape(new, (samples, m))
gen_df = gen_df.drop(columns=['Activity Type'])

plot_habits(gen_df)